# Time Series Forecasting

In [111]:
import numpy as np
import pandas as pd
import json
from matplotlib import pyplot as plt
from datetime import datetime
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.vector_ar.vecm import coint_johansen

In [115]:

coin_file = open("../../data/raw/coinmarketcap/historical_data.json")
coin_data = json.load(coin_file)

coin_data = pd.DataFrame(coin_data["Bitcoin"])
sentiment_data_df = pd.read_json("../../data/processed/reddit_summary_all.json", orient="index")

keyword_sentiment_data = sentiment_data_df["keyword_based_sentiment"].apply(pd.Series)

Bitcoin_specific_sentiment = keyword_sentiment_data["bitcoin"].apply(pd.Series)[["count","sentiment"]].dropna()

coin_data["timestamp"] = pd.to_datetime(coin_data["timestamp"]).dt.date

Bitcoin_specific_sentiment.reset_index(inplace=True)
Bitcoin_specific_sentiment.rename(columns = {"index": 'timestamp'}, inplace=True)
Bitcoin_specific_sentiment["timestamp"] = pd.to_datetime(Bitcoin_specific_sentiment["timestamp"]).dt.date


merged_data = pd.merge(left=coin_data, right=Bitcoin_specific_sentiment, how='left', left_on='timestamp', right_on='timestamp').dropna()

merged_data.index = merged_data["timestamp"]
merged_data.drop(labels = "timestamp", axis = 1, inplace=True)

merged_data.head(5)

,open,high,low,close,volume,marketCap,count,sentiment
timestamp,,,,,,,,
2021-06-07,43273.054043,44470.604093,40468.787642,40562.109146,4.067521e+10,7.590364e+11,2.0,0.208
2021-06-09,40480.023706,45478.402017,39317.719107,45243.801406,6.538069e+10,8.473666e+11,1.0,0.495
2021-06-10,45297.585159,46418.728004,43360.496468,44397.482847,5.279258e+10,8.329211e+11,2.0,-0.276
2021-06-14,47443.153091,49712.902865,47115.292443,48838.464001,5.246744e+10,9.161922e+11,1.0,0.403
2021-06-15,49091.884823,50341.616585,48205.357763,49245.140328,5.636933e+10,9.192833e+11,1.0,0.000


In [116]:
coint_johansen(merged_data, -1, 1).eig

array([0.69878119, 0.50903889, 0.44064159, 0.27856811, 0.22696237,
       0.10489492, 0.0369286 , 0.00164803])

In [199]:
#creating the train and validation set
split_ind = int(0.8*(len(merged_data)))
train = merged_data[:split_ind]
valid = merged_data[split_ind:]

model = VAR(endog=train)
results = model.fit(maxlags=2, ic="aic")
results.summary()

# make prediction on validation


Index([2022-03-10, 2022-03-11, 2022-03-12, 2022-03-13, 2022-03-14, 2022-03-15,
       2022-03-16, 2022-03-17, 2022-03-18, 2022-03-19, 2022-03-20, 2022-03-21,
       2022-03-22, 2022-03-23, 2022-03-24, 2022-03-25],
      dtype='object', name='timestamp')


C:\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


AttributeError: 'str' object has no attribute 'shape'